In [135]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.metrics import accuracy_score


In [136]:
students = pd.read_csv("data\Students_Grading_Dataset_Biased.csv")
students["Attendance (%)"] = students["Attendance (%)"].fillna(students["Attendance (%)"].mean())
students["Assignments_Avg"] = students["Assignments_Avg"].fillna(students["Assignments_Avg"].mean())
students["Parent_Education_Level"] = students["Parent_Education_Level"].fillna("Not Reported")
students.drop(["Student_ID","First_Name","Last_Name","Email"],axis=1, inplace=True)
students.head()
# grades = students["Grade"]
# students.drop("Grade",axis=1,inplace=True)

,Gender,Age,Department,Attendance (%),Midterm_Score,Final_Score,Assignments_Avg,Quizzes_Avg,Participation_Score,Projects_Score,Total_Score,Grade,Study_Hours_per_Week,Extracurricular_Activities,Internet_Access_at_Home,Parent_Education_Level,Family_Income_Level,Stress_Level (1-10),Sleep_Hours_per_Night
0,Female,22,Engineering,52.29,55.03,57.82,84.220000,74.06,3.99,85.90,56.09,F,6.2,No,Yes,High School,Medium,5,4.7
1,Male,18,Engineering,97.27,97.23,45.80,74.798673,94.24,8.32,55.65,50.64,A,19.0,No,Yes,Not Reported,Medium,4,9.0
2,Male,24,Business,57.19,67.05,93.68,67.700000,85.70,5.05,73.79,70.30,D,20.7,No,Yes,Master's,Low,6,6.2
3,Female,24,Mathematics,95.15,47.79,80.63,66.060000,93.51,6.54,92.12,61.63,A,24.8,Yes,Yes,High School,High,3,6.7
4,Female,23,CS,54.18,46.59,78.89,96.850000,83.70,5.97,68.42,66.13,F,15.4,Yes,Yes,High School,High,2,7.1


In [137]:
def assign_grade(score):
    if score >= 90:
        return 0
    elif score >= 80:
        return 1
    elif score >= 70:
        return 2
    elif score >= 60:
        return 3
    else:
        return 4

students["Grade"] = students["Total_Score"].apply(assign_grade)

In [138]:
category_cols = students.select_dtypes(include=['object']).columns.drop('Grade', errors='ignore')
encoder = OneHotEncoder(drop='first', sparse_output=False)  
encoded_data = encoder.fit_transform(students[category_cols])

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(category_cols))

students.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

students.drop(columns=category_cols, inplace=True)
students = pd.concat([students, encoded_df], axis=1)


In [139]:

# df_train, df_test = train_test_split(students, test_size=0.2, random_state=42) 

In [140]:
# df_train_labels = df_train["Grade"]
grades = students["Grade"]
students.drop('Total_Score',axis=1,inplace=True)
students.drop("Grade",axis=1,inplace=True)
numericalCols = students.select_dtypes(include=['number']).columns
scaler = StandardScaler()
students[numericalCols] = scaler.fit_transform(students[numericalCols])
x_train,x_test,y_train, y_test = train_test_split(students,grades, test_size=0.2, random_state=42) 

In [141]:

# model = LogisticRegression()

# # Hyperparameter grid
# param_grid = [
#     # Primary tuning: penalty + C combinations
#     {
#         'penalty': ['l1', 'l2'],
#         'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
#         'solver': ['liblinear', 'saga']  # Solvers that support both penalties
#     },
    
#     # Secondary option: class_weight balancing
#     {
#         'penalty': ['l2'],
#         'C': [0.1, 1.0, 10.0],
#         'class_weight': ['balanced'],
#         'solver': ['lbfgs', 'liblinear', 'saga']
#     }
# ]

# # Grid search
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train, y_train)

# print("Best parameters:", grid_search.best_params_)

In [142]:
lmodel = LogisticRegression(max_iter=500, random_state=42)
lmodel.fit(x_train,y_train)
y_predicted=lmodel.predict(x_test)
print(f"test Accurracy: {accuracy_score(y_test, y_predicted) * 100}%")


test Accurracy: 18.9%
